### Description:

#### Goals:
- Build a forecasting model to predict future web traffic for approximately 145,000 Wikipedia pages.
- Investigate the effectiveness of LSTM neural networks in handling multiple time series for this prediction problem.

#### Assumptions:
- The model is expected to forecast web traffic for 7 days based on the historical traffic from last 90 days.

#### Data and project inspiration:
https://www.kaggle.com/competitions/web-traffic-time-series-forecasting/data

#### Features:
- Main feature:
    - Traffic history (number of visits)
- Time-related features:
    - Day of the week
    - Day of the year
- Page-specific features:
    - Median visits over the whole period
    - Page attributes: project, access, agent

Logarithmic Transformation and Normalization were applied to the number of visits and the median of visits.

*For further details refer to exploration.ipynb*

#### Model architecture:
The model consists of:
- Two stacked LSTM layers (32 and 16 units)
- A hidden Dense layer (16 units)
- A multi-output Dense layer (7 units)

I also explored some variations including a single LSTM, varying numbers of Dense layers and varying numbers of units.
Multiple experiments indicated that the stacked LSTM architecture with Dense layers demonstrate the best performance compared to other variations tested.

#### Loss:
I used Huber loss with an alpha parameter set to 0.25.  
This choice helped the model focus on traffic patterns while disregarding occassional outliers encountered frequently.  
The 0.25 parameter value was determined based on 3 times the median standard deviation of page visits to filter out these outliers.  

*For further details refer to exploration.ipynb*

#### Samples, epochs and batch size:
The model is continuously supplied with random 90-day samples (followed by a 7-day predicted period) extracted from each page's visit history. Therefore a training epoch differs slightly from the conventional interpretation.
During each epoch, the model trains using 'n' samples extracted from every page, ensuring all pages contribute to each epoch. Subsequent epochs involve new samples from different time periods, maintaining diversity in the training data.

The number of samples per page, along with the batch size, determines whether batches comprise samples from the same or different pages:  
- For instance, if BATCH_SIZE=16 and N_SAMPLES=1, each batch contains 16 inputs from distinct pages.
- Conversely, if BATCH_SIZE=8 and N_SAMPLES=8, all 8 inputs in the batch are from a single page.

I experimented with various batch sizes (from 8 to 128) and numbers of samples (from 1 to 8). Through hyperparameter tuning, n_samples=4 and batch_size=8 were identified as optimal values.  
Reducing the batch size to 8 notably enhanced the model's generalization, albeit at the cost of increased computation time.  
Simultaneously setting the number of samples to 4 enabled the model to capture page-specific patterns by having only two pages per batch with 4 samples each.  

Efforts to equate the batch_size to the number of samples resulted in overfitting issues.

#### Regularization:
I conducted experiments using dropout and L2 regularization techniques.
- L2 Regularization proved effective by directing the model to emphasize diverse features, aiding in learning complex patterns.
- Dropout layers and LSTM's recurrent dropout didn't outperform L2 regularization and significantly increased the learning time. As a result, I chose to utilize L2 regularizatione exclusively.
- In hyperparameter tuning kernel regularization hindered the model's learning process. In contrast, recurrent regularization yielded the most favorable outcomes. While bias regularization didn't significantly impact the model's performance, I decided to employ recurrent regularization for its consistent results.

#### Optimizer and Learning Rate:
I employed the standard Adam optimizer with a decaying learning rate strategy. This approach aimed to swiftly achieve satisfactory results and subsequently focus on capturing additional patterns.

#### Hardware:
Hyperparameter tuning and the final model optimization were performed using A100 GPUs, resulting in a 90% reduction in computation time compared to CPU-based calculations.

In [ ]:
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from project_functions.sample_feed_v0_single import SampleFeed

In [ ]:
# Global parameters

TRAINING_WINDOW_SIZE = 90
PREDICTED_WINDOW_SIZE = 7
N_SAMPLES = 4
N_EPOCHS = 30
BATCH_SIZE = 8

In [ ]:
today_label = datetime.today().strftime("%m%d")

# Raw data

features_train = dict(np.load("data/features_train.npz", allow_pickle=True))
features_valid = dict(np.load("data/features_valid.npz", allow_pickle=True))

# Calculated parameters

n_rows_train = features_train['visits'].shape[0]
n_features = features_train['time'].shape[1] + features_train['page'].shape[1] + 1

steps_per_epoch = round(n_rows_train * N_SAMPLES / BATCH_SIZE)
total_samples_per_page = N_SAMPLES * N_EPOCHS

In [ ]:
# Sample Feed

sample_feed = SampleFeed(
    training_window_size = TRAINING_WINDOW_SIZE,
    predicted_window_size = PREDICTED_WINDOW_SIZE,
    samples_per_epoch = N_SAMPLES,
    n_features = n_features
    )

In [ ]:
# Prepare data

Xy_train_gen = sample_feed.random_sample_stream(features_train)
Xy_valid = sample_feed.random_sample_array(features_valid, samples_per_page=1, shuffle=False, seed=0)

In [ ]:
from keras import Sequential
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers
from keras import callbacks
from keras import regularizers

model = Sequential()
model.add(layers.InputLayer(input_shape=(TRAINING_WINDOW_SIZE, n_features)))
model.add(layers.LSTM(
    units=64, 
    return_sequences=True, 
    recurrent_regularizer=regularizers.L2(0.01)
    ))
model.add(layers.LSTM(
    units=32, 
    return_sequences=False,
    recurrent_regularizer=regularizers.L2(0.01)
    ))
model.add(layers.Dense(
    units=16, 
    activation='relu'
    ))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(PREDICTED_WINDOW_SIZE, 'sigmoid'))

model.compile(
    loss=losses.Huber(0.25), 
    optimizer=optimizers.Adam(learning_rate=1e-3), 
    metrics=metrics.RootMeanSquaredError()
    )

model.summary()

model_callbacks = [
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, min_lr=1e-5),
    callbacks.EarlyStopping(monitor='val_loss', patience=8),
    callbacks.ModelCheckpoint(filepath=f"models/checkpoints/{today_label}" + "{epoch:02d}-{val_root_mean_squared_error:.4f}.keras", monitor='val_loss')
]

In [ ]:
model_history = model.fit(
    x = Xy_train_gen,
    validation_data = Xy_valid,
    steps_per_epoch = steps_per_epoch,
    epochs = N_EPOCHS,
    batch_size = BATCH_SIZE,
    callbacks = model_callbacks
    )

In [ ]:
model.save(f"models/best_{today_label}", overwrite=False)

In [ ]:
plt.plot(model_history.history['loss'], color='black')
plt.plot(model_history.history['val_loss'], color='blue')